<a href="https://colab.research.google.com/github/UrielMendoza/LANOT_USAC_taller_sentinel2/blob/main/ejemplo_API_rest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>